## Questão 1

In [174]:
import numpy as np

def escalona_inteira(A):
    m, n = len(A), len(A[0])
    n_pivos = 0
    j = 0

    while j < n and n_pivos < m:
        L = [i for i in range(n_pivos, m) if A[i][j] != 0]

        while len(L) >= 2:
            p = min(L, key=lambda i: abs(A[i][j]))

            if A[p][j] < 0:
                for x in range(n):
                    A[p][x] *= -1

            for k in L:
                if k != p:
                    if A[k][j] > 0:
                        for i in range(n):
                            A[k][i] -=A[p][i]
                    else:
                        for i in range(n):
                            A[k][i] += A[p][i]

            L = [i for i in range(n_pivos, m) if A[i][j] != 0]

        if len(L) == 1:
            p = L[0]
            n_pivos += 1

        j += 1

    return A

In [175]:
M = [[2, 3, 5, -6],
     [0, -9, 1, 4],
     [7, 0, -2, 3],
     [-8, 3, 0, 4]]
print(np.linalg.det(M))
escalona_inteira(M)

2580.0000000000005


[[1, 12, 22, -27], [0, 3, -22, 12], [0, 0, 1, -424], [0, 0, 0, 860]]

## Questão 2

## Questão 3

## Questão 3.5

In [176]:
def DETERMINANTE_LAPLACE (A):
    n = len(A)

    det = 0

    if n == 1:
        return A[0][0]
    
    for j in range(n):
        A_j = []
        for a in range (1, n):
            lista = []
            for b in range(n):
                if b != j:
                    lista.append(A[a][b])
            A_j.append(lista)
        det += (-1)**j * A[0][j] * DETERMINANTE_LAPLACE(A_j)
    
    return det

In [177]:
DETERMINANTE_LAPLACE(M)

2580

## Questão 4